In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import cv2 as cv2
import numpy as np

import torch
import torch.hub

import torchvision
import torchvision.utils

from data import dataset as data
import data.utils as data_utils
import attack_construction.attack_methods as attack_methods
import attack_construction.metrics as metrics
import attack_construction.utils as attack_utils

print(torch.__version__)
print(torch.cuda_version)
print(torchvision.__version__)

1.10.0+cu113
11.3
0.11.1+cu113


In [2]:
device = torch.device("cpu")

#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
model.eval()
model = model.float().to(device)

dataset = data.AdversarialDataset() # todo: use resize to pull picture in batch

In [3]:
patch = attack_utils.load_tensor("patch - new")

In [14]:
image, label = dataset[5]

patch = torch.rand(3, 1000, 4)

image_before, image_after = attack_utils.get_patch_test(patch, image, label, model, device, None, 0.5)

cv2.imshow("patch", cv2.cvtColor(np.concatenate((image_before/255, image_after/255), axis = 1), cv2.COLOR_RGB2BGR))

key = cv2.waitKey(0)
if key & 0xFF == ord('q'):
    cv2.destroyAllWindows()

In [5]:
image, label = dataset[5]
pred_label = model([data_utils.image_to_tensor(image)])[0]


C:\Users\Степан\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [18]:
print(label)

tensor([[103.3200, 133.4500, 231.0300, 421.8900],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000]], dtype=torch.float64)


In [41]:
print(pred_label['scores'])

tensor([0.9512, 0.8752, 0.8034, 0.7582, 0.7398, 0.6379, 0.6187, 0.5611, 0.5536,
        0.5497, 0.4910, 0.4495, 0.4459, 0.4458, 0.4436, 0.4421, 0.3846, 0.3770,
        0.3603, 0.3546, 0.3243, 0.2736, 0.2644, 0.2584, 0.2577, 0.2550, 0.2480,
        0.2450, 0.2309, 0.2288, 0.2191, 0.2105, 0.2077, 0.1965, 0.1937, 0.1935,
        0.1928, 0.1863, 0.1857, 0.1797, 0.1781, 0.1781, 0.1779, 0.1734, 0.1702,
        0.1660, 0.1554, 0.1507, 0.1502, 0.1493, 0.1489, 0.1482, 0.1481, 0.1467,
        0.1444, 0.1427, 0.1423, 0.1410, 0.1408, 0.1390, 0.1367, 0.1342, 0.1342,
        0.1340, 0.1337, 0.1325, 0.1320, 0.1291, 0.1286, 0.1282, 0.1281, 0.1270,
        0.1264, 0.1264, 0.1263, 0.1261, 0.1259, 0.1223, 0.1211, 0.1208, 0.1208,
        0.1201, 0.1183, 0.1178, 0.1164, 0.1159, 0.1157, 0.1149, 0.1141, 0.1134,
        0.1133, 0.1127, 0.1125, 0.1123, 0.1117, 0.1103, 0.1099, 0.1094, 0.1078,
        0.1067, 0.1067, 0.1054, 0.1053, 0.1051, 0.1045, 0.1043, 0.1031, 0.1029,
        0.1028, 0.1019, 0.1016, 0.1015, 

In [38]:
metrics.intersection_over_union(pred_label['boxes'][0].detach(), label[0].detach())

tensor([0.9078], dtype=torch.float64)

In [45]:
dataset = data.AdversarialDataset((800,300))
dataset_val = data.AdversarialDataset((640,640), '../../val2017/val2017', '../../annotations_trainval2017/annotations/instances_train2017.json')
train_loader  = torch.utils.data.DataLoader(dataset=dataset, batch_size=3, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.Subset(dataset_val,5), batch_size=3, shuffle=False)
def loss_function(predict, patch, device):
        return metrics.general_objectness(predict, device) + 0 * metrics.total_variation(patch)

KeyboardInterrupt: 